In [1]:
import torch

print(torch.__version__)

2.0.0+cu117


In [2]:
!pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 795.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchaudio 2.0.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 KB 16.2 MB/s eta 0:00:00
  Using cached torch-2.0.0-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install --upgrade jupyter ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 KB 8.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
import timm

In [6]:
# 데이터셋 경로 설정 및 전처리
data_dir = '/kaggle/input/products-10k'
train_csv_file = data_dir + '/train.csv'
test_csv_file = data_dir + '/test.csv'

In [7]:
import pandas as pd

# train.csv 파일 로드
train_data = pd.read_csv(train_csv_file)
test_data = pd.read_csv(test_csv_file)

# 컬럼 이름 출력
print(train_data.columns)
print(test_data.columns)

Index(['name', 'class', 'group'], dtype='object')
Index(['name'], dtype='object')


In [8]:
# 전처리를 위한 변환 설정
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # 모델에 맞는 입력 크기로 조정
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
])

In [9]:
# 사용자 정의 데이터셋 클래스 정의
class Products10kDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None, is_train=True):
        self.data = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.transform = transform
        self.is_train = is_train
        
        if self.is_train:
            self.classes = self.data['class'].unique().tolist()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['name']
        if self.is_train:
            img_path = f"{self.data_dir}/train/train/{img_path}"
        else:
            img_path = f"{self.data_dir}/test/test/{img_path}"
        image = Image.open(img_path).convert('RGB')

        if self.is_train:
            label = self.data.iloc[idx]['class']
        else:
            label = None  # 테스트 데이터셋에는 레이블이 없으므로 None으로 설정

        if self.transform:
            image = self.transform(image)

        return image, label


In [10]:
# 데이터셋 불러오기
train_dataset = Products10kDataset(train_csv_file, data_dir, transform=transform, is_train = True)
test_dataset = Products10kDataset(test_csv_file, data_dir, transform=transform, is_train = False)

In [11]:
# 데이터 로더 설정
batch_size = 32
num_workers = 1  # TPU를 사용하기 때문에 num_workers를 1로 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

In [12]:
# 모델 생성 및 사전 훈련된 가중치 로드
model = timm.create_model('cait_s36_384', pretrained=True)
num_classes = len(train_dataset)
model.head = nn.Linear(model.head.in_features, num_classes)  # 분류를 위한 마지막 레이어 변경

In [13]:
!pip install torch-xla

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [14]:
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl

# TPU 디바이스 설정
device = xm.xla_device()

# 모델을 TPU 디바이스로 이동
model = model.to(device)

# TPU를 위한 데이터 로더로 변경
train_loader = pl.MpDeviceLoader(train_loader, device)
test_loader = pl.MpDeviceLoader(test_loader, device)

# 모델을 분산 처리할 수 있도록 설정
model = nn.DataParallel(model)

E0521 03:57:24.583005896      14 oauth2_credentials.cc:236]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2023-05-21T03:57:24.582989001+00:00", grpc_status:2}


In [15]:
# 손실 함수 설정
criterion = nn.CrossEntropyLoss().to(device)

# 최적화 알고리즘 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# 학습 및 평가 함수 정의
def train(model, dataloader, criterion, optimizer):
    model.train()
   
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        xm.optimizer_step(optimizer)
        
        running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy

def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy


In [17]:
# 모델 학습 및 평가
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion)

    print(f"Epoch: {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")
    print()
    

KeyboardInterrupt: 

In [2]:
!pip install vit-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR100
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from vit_pytorch.cait import CaiT

#전처리를 위한 변환 설정
transform = transforms.Compose([
transforms.Resize((32, 32)), # CIFAR-100 이미지 크기로 조정
transforms.ToTensor(), # 이미지를 Tensor로 변환
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 이미지 정규화
])

#CIFAR-100 데이터셋 불러오기
train_dataset = CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = CIFAR100(root='./data', train=False, transform=transform, download=True)

#데이터 로더 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

#모델 생성
model = CaiT(
image_size=32,
patch_size=4,
num_classes=100,
dim=1024,
depth=12,
cls_depth=2,
heads=16,
mlp_dim=2048,
dropout=0.1,
emb_dropout=0.1,
layer_dropout=0.05
)

#디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#학습 및 평가 함수 정의
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy

def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0


    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            running_loss += loss.item()

    accuracy = 100 * correct / total
    return running_loss, accuracy
#모델 학습 및 평가
num_epochs = 10

for epoch in range(num_epochs):
    print("Trainning step: ", epoch)
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    print("Evaluating step: ", epoch)
    val_loss, val_accuracy = evaluate(model, test_loader, criterion)

    print("Epoch: {}/{}".format(epoch+1, num_epochs))
    print("Train Loss: {:.4f} | Train Accuracy: {:.2f}%".format(train_loss, train_accuracy))
    print("Val Loss: {:.4f} | Val Accuracy: {:.2f}%".format(val_loss, val_accuracy))
    print()


Files already downloaded and verified
Files already downloaded and verified
Trainning step:  0
Evaluating step:  0
Epoch: 1/10
Train Loss: 7308.5967 | Train Accuracy: 1.41%
Val Loss: 1441.8101 | Val Accuracy: 1.46%

Trainning step:  1
Evaluating step:  1
Epoch: 2/10
Train Loss: 7101.3206 | Train Accuracy: 1.84%
Val Loss: 1418.9964 | Val Accuracy: 1.72%

Trainning step:  2
Evaluating step:  2
Epoch: 3/10
Train Loss: 7071.1068 | Train Accuracy: 1.85%
Val Loss: 1416.5116 | Val Accuracy: 1.70%

Trainning step:  3
Evaluating step:  3
Epoch: 4/10
Train Loss: 7033.7720 | Train Accuracy: 1.99%
Val Loss: 1422.4393 | Val Accuracy: 2.09%

Trainning step:  4
Evaluating step:  4
Epoch: 5/10
Train Loss: 7062.1633 | Train Accuracy: 1.96%
Val Loss: 1429.5803 | Val Accuracy: 1.85%

Trainning step:  5
Evaluating step:  5
Epoch: 6/10
Train Loss: 7083.7138 | Train Accuracy: 1.97%
Val Loss: 1417.4810 | Val Accuracy: 1.69%

Trainning step:  6
Evaluating step:  6
Epoch: 7/10
Train Loss: 7054.5863 | Train Acc